In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
df = pd.read_csv('vehicles_no_url_des.csv', header=0, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'vehicles_no_url_des.csv'

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
manufacturer_count = pd.DataFrame(columns=['counts','brands'])
manufacturer_count['counts'] = df.groupby(['manufacturer']).count().sort_values('url')[-25:]['url']
manufacturer_count['brands'] = df.groupby(['manufacturer']).count().sort_values('url')[-25:].index

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10,10))
X = np.arange(len(manufacturer_count))
ax1.barh(X, manufacturer_count['counts'], tick_label = manufacturer_count['brands'])
plt.show()

In [ ]:
fig = px.pie(manufacturer_count, values='counts', names='brands', title='Number of cars by manufacturer')
fig.show()

In [ ]:
df_nonull_prices = df.dropna(subset=['price'], axis='index')
p = df_nonull_prices.groupby(['state']).mean().price
statewise_prices = pd.DataFrame({'states':p.index, 'avg_price':p.values}) 
statewise_prices['states'] = statewise_prices['states'].str.upper()

In [ ]:
#TODO - split into groups of price values or percentiles to distinguish easily
fig = px.choropleth(statewise_prices, locations='states', locationmode="USA-states",  
                           color='avg_price',
                           #color_continuous_scale="Viridis",
                           scope="usa",
                           color_continuous_scale=px.colors.sequential.Plasma
                          )
fig.show()

In [ ]:
df.model.isna().count() / df.model.count()

In [ ]:
np.count_nonzero(df.model.isna()) / len(df.model)

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime
import dateutil

from numpy.random import default_rng
import requests
from IPython.display import Image, HTML
rng = default_rng()

In [ ]:
test_model = rng.choice(df[["manufacturer", "model"]])
test_model = " ".join(test_model)
test_model

In [5]:
#TODO: eventually migrate this client side.

def make_wikimedia_request(api_url="https://en.wikipedia.org/w/api.php", **parameters):
    r =  requests.get(api_url, params = dict(format="json", **parameters))
    assert r.status_code == 200
    return r.json()


def wikimedia_search(query, **parameters):
    return make_wikimedia_request(
        action="opensearch", 
        prop="images",
        imlimit=1,
        search=query,
        **parameters
        )

def wikipedia_search(query, **parameters):
    return make_wikimedia_request(
        action="opensearch", 
        limit=1,
        search=query,
        **parameters
        )



In [6]:
r = wikipedia_query_firstpage(test_model)
r

NameError: name 'wikipedia_query_firstpage' is not defined

In [ ]:
@dataclass
class WikipediaPage():
    ns: int
    title: str
    pageid: int
    size: int
    wordcount: int
    snippet: str = field(repr=False)
    timestamp: datetime
        
    def __post_init__(self):
        if isinstance(self.timestamp, str):
            self.timestamp = dateutil.parser.parse(self.timestamp)
    
    @classmethod
    def from_query(cls, q:str):
        r = wikipedia_query_firstpage(q)
        if r is None:
            return None
        return cls(**r)
    
   
    def get_thumbnail(self):
        r = make_wikimedia_request(action="query", 
                                   prop="pageimages", 
                                   piprop="original", 
                                   pageids=self.pageid,
                                  pilicense="free")
        pages = r["query"]["pages"]
        _, onlypage = pages.popitem()
        file = onlypage.get("original", None)
        return file
    
    def display_thumbnail(self):
        """Displays the thumbnail in a jupyter notebook"""
        thumbnail = self.get_thumbnail()["source"]
        display(Image(thumbnail))
        
    def get_quicktext(self):
        r = make_wikimedia_request(action="query", 
                           prop="extracts",
                            exintro="true", 
                           pageids=self.pageid,
                          pilicense="free")
        pages = r["query"]["pages"]
        _, onlypage = pages.popitem()
        extract = onlypage.get("extract", None)

        return extract

        
        
display(WikipediaPage.from_query("Truth").get_thumbnail())
display(WikipediaPage.from_query(test_model).get_thumbnail())
WikipediaPage.from_query(test_model).get_quicktext()

In [ ]:
WikipediaPage.from_query("bmw i3").display_thumbnail()

In [ ]:
HTML(WikipediaPage.from_query(test_model).get_quicktext())

https://commons.wikimedia.org/w/api.php?action=opensearch&prop=images&imlimit=1&search=Felis_silvestris_catus

In [8]:
import sys
sys.path.append("..")

In [17]:
from .wikipedia_api import WikipediaPage

In [18]:
page = WikipediaPage.from_query("bmw i3")

In [21]:
page.get_thumbnail()

{'source': 'https://upload.wikimedia.org/wikipedia/commons/9/98/2018_BMW_i3_facelift_%281%29.jpg',
 'width': 4608,
 'height': 2829}

In [22]:
page.get_quicktext()

'<p class="mw-empty-elt">\n</p>\n\n\n<p>The <b>BMW i3</b> is a B-segment, high-roof hatchback manufactured and marketed by BMW with an electric powertrain using rear wheel drive via a single-speed transmission and an underfloor Lion battery pack and an optional range-extending petrol engine. The i3 was BMW\'s first mass-produced zero emissions vehicle and was launched as part of BMW\'s electric vehicle <i>BMW i</i> sub-brand.</p><p>Styled by Richard Kim, the i3 is offered in a single five-door configuration with a passenger module of high strength, ultra-lightweight CFRP (carbon fibre reinforced polymer) adhered to a lower aluminium module holding the chassis, battery, drive system and powertrain. The bodywork features two smaller clamshell rear-hinged rear doors.\n</p><p>The i3 debuted as a concept at the 2011 International Motor Show Germany, and production began in September 2013 in Leipzig.\n</p><p>Having ranked third amongst electric cars sold worldwide from 2014 to 2016, its glob